In [26]:
!pip install PyPDF2
!pip install python-docx
!pip install find-job-titles
!pip install Levenshtein
import nltk
nltk.download('stopwords')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.1/383.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 14.5 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

We used this code to read the multiple documents and extract the text.

In [27]:
from bs4 import BeautifulSoup
from docx import Document
from PyPDF2 import PdfReader

def identify_file_type(file_path):
    resume = ""

    file_name = file_path.split('/')[-1]  # Extracting the file name from the path
    if file_name.endswith('.pdf'):
        reader = PdfReader(file_path)
        for i in range(len(reader.pages)):
            page = reader.pages[i]
            text = page.extract_text()
            resume += text
        return resume

    elif file_name.endswith('.docx'):
        document = Document(file_path)
        for p in document.paragraphs:
            resume += p.text
        return resume

    elif file_name.endswith('.html'):
        with open(file_path, 'r', encoding='utf-8') as html_file:
            soup = BeautifulSoup(html_file, 'html.parser')
            for line in soup.get_text().splitlines():
                resume += line
        return resume

    else:
        return 'Unknown File Type'

# Example usage:
file_path = "/content/documents20220826-1-v01cla.pdf"  # Example file path
resume = identify_file_type(file_path)
print("File type:", resume)

File type: Ashwini Kalidas Vedula
Email ID: ashwini.vedula@nyu.edu |+1 (201)-616-1347 |LinkedIn: linkedin.com/in/ashwini-vedula
Education
New York University, New York, NY Sept 2021- May 2023
Courant Institute of Mathematical Sciences and Stern School of Business
Master of Science in Information Systems - 3.6/4
Courses: Dealing with Data, Data Science for Business : Technical, Database Management Systems, Real Time
Big Data Analytics, Cloud Computing
Technical Skills
Databases & Tools : MySQL, SSMS, MS Access, PostgreSQL, PL/SQL, MongoDB
Data Analysis & Visualization Tools : Python, R , SQL, MS Excel, Tableau, Power BI
Cloud Platforms & Big Data Technologies : AWS (Redshift, EC2, S3, EMR, SageMaker, DynamoDB Table), Azure
(Storage, IoT, Virtual Machines), GCP (BigTable, DataLab), Apache Hadoop, Hive
Experience
Clark Associates Inc Pennsylvania, USA
Financial Systems and Data Intern May 2022 - Aug 2022
•Extracted data from varied data sources to automate Summary Income financial reports

This code is for creating the ONET dataset. The input is the original dataset and the output is a dataset with only the umbrella titles (those whose code ends with 00) included

In [29]:
import pandas as pd
import re

# ONET dataframe
df = pd.read_csv("2019_Occupations.csv")

# Function to check if a numeric code ends with '00'
def ends_with_00(code):
    return re.match(r'.*00$', str(code)) is not None

# Filter dataframe to find strings where their associated numeric codes end with '00'
ONET_final = df[df['O*NET-SOC 2019 Code'].apply(ends_with_00)]
ONET_final.to_csv("ONET_final.csv")

we then ran this code for each resume and used the extracted job titles to create our initial job title dataset.

In [30]:
from find_job_titles import Finder
import re
finder = Finder()
def find_jobs(resume):
  return(list(finder.findall(resume)))

def extract_job_strings(jobs):
  f_jobs = []
  job_pattern = r"'([^']*)'"
  for job in jobs:
    j = str(job)
    job_extracted = str(re.findall(job_pattern, j))
    if (job_extracted not in f_jobs):
      f_jobs.append(job_extracted)
    else:
      continue
  return f_jobs

def remove_extra_characters(input_string):
  cleaned_string = re.sub(r'\W+', ' ', input_string)
  return cleaned_string.strip()

jobs_from_resume = find_jobs(resume)
f_jobs = extract_job_strings(jobs_from_resume)
final_jobs = []
for j in f_jobs:
  final_jobs.append(remove_extra_characters(j))

print(final_jobs)

['Associate', 'Senior Analyst', 'Senior Associate', 'Business Development', 'Associate Software Engineer', 'Student']


We split the dataset into single word and multi word job title **dataframes**

In [33]:
# Splitting the dataframe based on single and multiple word job titles
df = pd.read_csv("job_titles_resume.csv")
single_word_df = df[df['Job Title'].apply(lambda x: len(x.split()) == 1)]
multiple_word_df = df[df['Job Title'].apply(lambda x: len(x.split()) > 1)]

# Save dataframes to separate CSV files
single_word_df.to_csv('single_word_job_titles.csv', index=False)
multiple_word_df.to_csv('multiple_word_job_titles.csv', index=False)

print("Dataframes have been split and saved as CSV files.")

Dataframes have been split and saved as CSV files.


Levenshtein distance is used to map the single word job titles to the ONET job titles

In [34]:
from Levenshtein import distance
import pandas as pd

# Sample job titles from resumes
resume_jobs = pd.read_csv("/content/single_word_job_titles.csv")
resume_job_titles = resume_jobs["Job Title"].tolist()

# Sample job titles from ONET database
onet_jobs = pd.read_csv("ONET_final.csv")
onet_job_titles = onet_jobs["O*NET-SOC 2019 Title"].tolist()

# Function to find the most similar job title using Levenshtein distance
def find_most_similar(resume_title, onet_titles):
    min_distance = float('inf')
    most_similar_title = None
    for onet_title in onet_titles:
        dist = distance(resume_title.lower(), onet_title.lower())
        if dist < min_distance:
            min_distance = dist
            most_similar_title = onet_title
    return most_similar_title

# Mapping of resume job titles to most similar ONET job titles
resume_to_onet_mapping = {}
for resume_title in resume_job_titles:
    most_similar_onet_title = find_most_similar(resume_title, onet_job_titles)
    resume_to_onet_mapping[resume_title] = most_similar_onet_title

# Convert the dictionary to a DataFrame
mapping_df = pd.DataFrame(resume_to_onet_mapping.items(), columns=['Resume Job Title', 'Most Similar ONET Job Title'])

# Save the DataFrame to a CSV file
mapping_df.to_csv('single_job_mappings.csv', index=False)

print("Mapping saved to single_job_mappings.csv")


Mapping saved to single_job_mappings.csv


We used BERT to map the multi word job titles to the ONET job titles.

In [37]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define the dictionary of job titles and their corresponding full forms
job_titles = {
    "CEO": "Chief Executive Officer",
    "CTO": "Chief Technology Officer",
    "CFO": "Chief Financial Officer",
    "COO": "Chief Operating Officer",
    "CMO": "Chief Marketing Officer",
    "CIO": "Chief Information Officer",
    "CSO": "Chief Security Officer"
}

# Function to generate BERT embeddings for a given text
def get_bert_embeddings(text):
    # Define words to exclude from embeddings
    excluded_words = ["intern"]

    # Remove excluded words from the text
    cleaned_text = ' '.join(word for word in text.split() if word.lower() not in excluded_words)

    # Tokenize the cleaned text
    inputs = tokenizer(cleaned_text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract embeddings from BERT's output
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Function to compute cosine similarity between two vectors
def compute_cosine_similarity(vector1, vector2):
    return cosine_similarity(vector1.reshape(1, -1), vector2.reshape(1, -1))[0][0]

# Load job titles from CSV files
resume_data = pd.read_csv('/content/multiple_word_job_titles.csv')
onet_data = pd.read_csv('/content/ONET_final.csv')

# Extract job titles from dataframes
resume_job_titles = resume_data['Job Title'].tolist()
onet_job_titles = onet_data['O*NET-SOC 2019 Title'].tolist()

# Generate BERT embeddings for job titles
resume_embeddings = {title: get_bert_embeddings(title) for title in resume_job_titles}
onet_embeddings = {title: get_bert_embeddings(title) for title in onet_job_titles}

# Map each resume job title to ONET job titles based on similarity
job_title_mapping = {}
for resume_title, resume_embedding in resume_embeddings.items():
    # Check if the job title contains "software"
    if 'software' in resume_title.lower():
        job_title_mapping[resume_title] = "Software Developers"
    # Check if the resume title corresponds to any key in the dictionary
    elif any(key in resume_title for key in job_titles.keys()):
        # If yes, map it to "Chief Executive"
        job_title_mapping[resume_title] = "Chief Executive"
    else:
        similarity_scores = {}
        for onet_title, onet_embedding in onet_embeddings.items():
            similarity_scores[onet_title] = compute_cosine_similarity(resume_embedding, onet_embedding)
        # Find the ONET job title with the highest similarity score
        best_match = max(similarity_scores, key=similarity_scores.get)
        job_title_mapping[resume_title] = best_match

# Convert the mapping dictionary to a DataFrame
mapping_df = pd.DataFrame(job_title_mapping.items(), columns=['Resume Job Title', 'ONET Job Title'])

# Save the DataFrame to a CSV file
mapping_df.to_csv('job_title_mapping.csv', index=False)
print("mapping saved to job_title_mapping.csv")

mapping saved to job_title_mapping.csv


We then merged the two mapped dataframes together and further merged it with the ONET codes, creating out final dataset ONET_codes. This contains the resume job title, the onet matched job title and the respective code.

In [42]:
import pandas as pd

# Read the CSV files
final_job_mapping = pd.read_csv("/content/Final Job Mapping.csv")
onet_final = pd.read_csv("/content/ONET_final.csv")

# Merge the two dataframes based on the common column "ONET Job Title"
merged_df = pd.merge(final_job_mapping, onet_final, left_on="ONET Job Title", right_on="Job Titles", how="left")

# Drop the redundant "Job Titles" column
merged_df.drop(columns=["Job Titles"], inplace=True)

# Save the merged dataframe to a new CSV file
# merged_df.to_csv("merged.csv", index=False)
merged_df = merged_df.drop(["Unnamed: 0", "Job Descriptions"], axis=1)
merged_df.to_csv("ONET_codes.csv")